# Missionaries and Cannibals

The Missionaries and Cannibals problem is usually stated as follows. Three missionaries and there cannibals are on one side of a river, along with a boat that can hold one or two people. Find a way to get everyone to the other side without ever leaving a group of missionaries in one place outnumbered by the cannibals in that place.

In [24]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['/Users/mikezhu/Dev/Python/aima-python'])

Python 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)] on darwin




## Goal Formulation

The goal for this problem is to find a stratege for the three missionaries and cannibals to cross the river with a boat which can take at most two people, and make sure at every step no missionary at one side is facing more cannibals than missionaries. We assume all the people starts at the RIGHT side of the river.

## Formulating the Problem

### State

We can describe the states using a tuple contains these information:
* The number of missionaries on the left side of the river.
* The number of cannibals on the left side of the river.
* The number of missionaries on the right side of the river.
* The number of cannibals on the right side of the river.
* Where's the boat.

So it's proper to formulate a tuple with 5 numbers:
```
(0, 0, 3, 3, 1)
```
where the numbers obey the sort as presented above. The last number equals 1 indicating the boat is on the right side of the river and 0 indicating on the left side.

So the starting state and the goal state can be presented as:


In [25]:
starting_state = (0, 0, 3, 3, 1)
goal_state = (3, 3, 0, 0, 0)


### Actions

At each state, we have actions to drive one or two people to the other side of the river. Take the side into consideration, we have ten actions:


In [26]:
all_possible_actions = ['ML', 'MR', 'CL', 'CR', 'MML', 'MMR', 'CCL', 'CCR', 'MCL', 'MCR']


In these actions, 'ML' means the boat take one missionary on the left side to the right side, 'MMR' means the boat will take two missionaries on the right side to the left side, 'MCR' indicating one missionary and one cannibal will be taken from right side to the left, and others are similarly defined.

However, not all the actions can be excute at one state. The straightforward issue is when the boat is on the right side of the river, actions which take people from left side to the right side are not applicatble. Other issues are listed in the code below.

In [27]:
from search import Problem

class MissionariesAndCannibals(Problem):
    """"""
    def __init__(self, initial, goal=goal_state):
        """ Define goal state and initialize a problem
        (3, 3, 0, 0, 0)
        (M, C, M, C, B)
        """

        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, state):
        possible_actions = all_possible_actions.copy()
        is_right_side = (state[-1] == 1)
        if is_right_side:
            possible_actions.remove('ML')
            possible_actions.remove('CL')
            possible_actions.remove('MML')
            possible_actions.remove('CCL')
            possible_actions.remove('MCL')

            # MR
            if (state[2] - 1 < 0) or (0 < state[2] - 1 < state[3]) or (0 < state[0] + 1 < state[1]):
                possible_actions.remove('MR')

            # MMR
            if (state[2] - 2 < 0) or (0 < state[2] - 2 < state[3]) or (0 < state[0] + 2 < state[1]):
                possible_actions.remove('MMR')

            # MCR
            if (state[2] - 1 < 0) or (state[3] - 1 < 0) or (0 < state[0] + 1 < state[1] + 1):
                possible_actions.remove('MCR')

            # CR
            if (state[3] - 1 < 0) or (0 < state[0] < state[1] + 1):
                possible_actions.remove('CR')

            # CCR
            if (state[3] - 2 < 0) or (0 < state[0] < state[1] + 2):
                possible_actions.remove('CCR')

        else:
            possible_actions.remove('MR')
            possible_actions.remove('CR')
            possible_actions.remove('MMR')
            possible_actions.remove('CCR')
            possible_actions.remove('MCR')

            # ML
            if (state[0] - 1 < 0) or (0 < state[0] - 1 < state[1]) or (0 < state[2] + 1 < state[3]):
                possible_actions.remove('ML')

            # MML
            if (state[0] - 2 < 0) or (0 < state[0] - 2 < state[1]) or (0 < state[2] + 2 < state[3]):
                possible_actions.remove('MML')

            # MCL
            if (state[0] - 1 < 0) or (state[1] - 1 < 0) or (0 < state[2] + 1 < state[3] + 1):
                possible_actions.remove('MCL')

            # CL
            if (state[1] - 1 < 0) or (0 < state[2] < state[3] + 1):
                possible_actions.remove('CL')

            # CCL
            if (state[1] - 2 < 0) or (0 < state[2] < state[3] + 2):
                possible_actions.remove('CCL')

        return possible_actions

    def result(self, state, action):
        new_state = list(state)

        if action == 'ML':
            new_state[0] = new_state[0] - 1
            new_state[2] = new_state[2] + 1

        elif action == 'MML':
            new_state[0] = new_state[0] - 2
            new_state[2] = new_state[2] + 2

        elif action == 'MCL':
            new_state[0] = new_state[0] - 1
            new_state[1] = new_state[1] - 1
            new_state[2] = new_state[2] + 1
            new_state[3] = new_state[3] + 1

        elif action == 'CL':
            new_state[1] = new_state[1] - 1
            new_state[3] = new_state[3] + 1

        elif action == 'CCL':
            new_state[1] = new_state[1] - 2
            new_state[3] = new_state[3] + 2

        elif action == 'MR':
            new_state[0] = new_state[0] + 1
            new_state[2] = new_state[2] - 1

        elif action == 'MMR':
            new_state[0] = new_state[0] + 2
            new_state[2] = new_state[2] - 2

        elif action == 'MCR':
            new_state[0] = new_state[0] + 1
            new_state[1] = new_state[1] + 1
            new_state[2] = new_state[2] - 1
            new_state[3] = new_state[3] - 1

        elif action == 'CR':
            new_state[1] = new_state[1] + 1
            new_state[3] = new_state[3] - 1

        elif action == 'CCR':
            new_state[1] = new_state[1] + 2
            new_state[3] = new_state[3] - 2

        new_state[4] = 1 - new_state[4]

        return tuple(new_state)

    def value(self, state):
        pass

    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """

        return state == self.goal

## Seaching for the Result

We do breadth first graph search for the solution:

In [28]:
from search import breadth_first_graph_search


def solve():
    missionaries_and_cannibals = MissionariesAndCannibals(starting_state)
    return breadth_first_graph_search(missionaries_and_cannibals).solution()


And we can get the action list to reach the goal.

In [29]:
print(solve())


['CCR', 'CL', 'CCR', 'CL', 'MMR', 'MCL', 'MMR', 'CL', 'CCR', 'ML', 'MCR']
